In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

# mnist = np.load("dataset/mnist-784-euclidean.npy")
# print(f'mnist data shape: {mnist.shape}')
# print(f'mnist dtype: {mnist.dtype}')

mnist = np.random.rand(60000, 2)
mnist = mnist.astype(np.float32)
print(f'mnist dtype: {mnist.dtype}')
print(f'mnist data shape: {mnist.shape}')


mnist dtype: float32
mnist data shape: (60000, 2)


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[    0 13281 47047 43292 35461 21260 44106 25403 46079 17353 19186 54316
   3755   107 49686 25207 30187 41100 11501 38087 47597 30828  4008 24209
  57977 15755 36121 55140 22845 15948  5540 57052 31558 42695 16225 40654
  30421  9198 31342  2280 20323 47625 11349 27604 35158 22772 29586 41355
   1511 49983 38871 35881 30627  9214 12911 35638 35193 31703 41079  8442
   5580 10867 37159 28156 14772  2887 13976 25923 48432 22056 56770 59987
  54988 50799 20578 43843 59416 11057 34942 46613 47389  7796 36661 43069
   6197 50012 57999 48168 37640 12389 57298 11551  7726 59651 31802  2598
  35864 51308 42830 40049]]
CPU times: total: 0 ns
Wall time: 999 µs


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

index = index.Indexing(
    gl_size=80000, 
    ll_size=2048,
    g_epoch_num=5,
    l_epoch_num=10,
    g_hidden_size=5,
    l_hidden_size=20,
    l_block_range=2,
    random_partitioning=False
)
index.train(mnist_tensor)

print(f"local model len:{len(index._l_model)}")

mnist tensor shape: torch.Size([60000, 2])
torch.Size([60000, 3])
first stage partitioning finish
partitioning blocks : 1
training local model


d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


1, 100 loss: 0.07258500576019287
1, 200 loss: 0.07912117004394531
1, 300 loss: 0.07160125732421875
1, 400 loss: 0.07943352222442628
1, 500 loss: 0.07968381881713867
1, 600 loss: 0.06827491283416748
2, 100 loss: 0.0745693826675415
2, 200 loss: 0.07299689769744873
2, 300 loss: 0.0753208589553833
2, 400 loss: 0.07741924285888672
2, 500 loss: 0.07756493091583253
2, 600 loss: 0.07867400169372558
3, 100 loss: 0.07385792255401612
3, 200 loss: 0.07945691585540772
3, 300 loss: 0.067811279296875
3, 400 loss: 0.0706406021118164
3, 500 loss: 0.07722878932952881
3, 600 loss: 0.07498417377471923
4, 100 loss: 0.07467264652252198
4, 200 loss: 0.0795082426071167
4, 300 loss: 0.07138856410980225
4, 400 loss: 0.07881863117218017
4, 500 loss: 0.0673963499069214
4, 600 loss: 0.07825123310089112
5, 100 loss: 0.0689461326599121
5, 200 loss: 0.08121406555175781
5, 300 loss: 0.08343879699707031
5, 400 loss: 0.06784330368041992
5, 500 loss: 0.08593755722045898
5, 600 loss: 0.06729098320007325
6, 100 loss: 0.072

d:\study\bspli\./Bspli\utils\Partitioning.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time
qp = torch.from_numpy(mnist[0])
# print(qp)
pred = index.query(qp, k=100)
pred = pred.to(torch.int)
print(f"pred: {pred}")

search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([60000, 2])
pred: tensor([    0, 13281, 47047, 43292, 35461, 21260, 44106, 25403, 46079, 17353,
        19186, 54316,  3755,   107, 49686, 25207, 30187, 41100, 11501, 38087,
        47597, 30828,  4008, 24209, 57977, 15755, 36121, 55140, 22845, 15948,
         5540, 57052, 31558, 42695, 16225, 40654, 30421,  9198, 31342,  2280,
        20323, 47625, 11349, 27604, 35158, 22772, 29586, 41355,  1511, 49983,
        38871, 35881, 3062

In [5]:
result = []

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size


def benchmark_knn_query(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
    result.append((query_time/1000, cur_recall/1000))

In [6]:
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, index, size=1000, k=10)

search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([60000, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total numbers: torch.Size([0, 2])
search total

In [7]:
print(result)

[(0.005117665767669677, 1.0)]
